In [1]:
import pandas as pd
import plotly.express as px
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import plotly.io as pio

df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')

df['dt'] = pd.to_datetime(df['dt']) #convert time
df['Year'] = df['dt'].dt.year

df = df.dropna(subset=['AverageTemperature']) #remove missing values

# Average temperature per year for each country
avg_temp_per_year = df.groupby(['Country', 'Year'])['AverageTemperature'].mean().reset_index()

# Average temperature for each country over all recorded years
avg_temp_change = avg_temp_per_year.groupby('Country')['AverageTemperature'].mean().reset_index()

# Change in temperature from the first to the last recorded year for each country
first_last_temp = avg_temp_per_year.groupby('Country')['AverageTemperature'].agg(['first', 'last']).reset_index()
first_last_temp['TempChange'] = first_last_temp['last'] - first_last_temp['first']

# First and last year for each country
first_last_temp['FirstYear'] = avg_temp_per_year.groupby('Country')['Year'].min().values
first_last_temp['LastYear'] = avg_temp_per_year.groupby('Country')['Year'].max().values

# Add first and last temperatures to the DataFrame
first_last_temp['FirstTemp'] = avg_temp_per_year.groupby('Country')['AverageTemperature'].first().values
first_last_temp['LastTemp'] = avg_temp_per_year.groupby('Country')['AverageTemperature'].last().values

# Initialize the Dash app
app = Dash(__name__)

# Map average yearly temperature by country
avg_temp_map_fig = px.choropleth(avg_temp_change,
                                   locations = 'Country',
                                   locationmode = 'country names',
                                   color = 'AverageTemperature',
                                   color_continuous_scale = px.colors.sequential.Turbo,
                                   range_color = [-20, 25],
                                   title = 'Average Yearly Temperature by Country')

# Generate a choropleth map for temperature change from the first to last year recorded
temp_change_map_fig = px.choropleth(first_last_temp,
                                     locations = 'Country',
                                     locationmode = 'country names',
                                     color = 'TempChange',
                                     color_continuous_scale = px.colors.sequential.Sunset,
                                     range_color = [-2, 12],
                                     title = 'Change in Average Yearly Temperature',
                                     hover_name = 'Country',
                                     hover_data = ['FirstYear', 'FirstTemp', 'LastYear', 'LastTemp'])



# Define the layout of the app
app.layout = html.Div([
    html.H1('Global Temperature Data Dashboard'),

    # Dropdown for country selection
    dcc.Dropdown(
        id = 'country-dropdown',
        options = [{'label': country, 'value': country} for country in avg_temp_change['Country']],
        value = 'United Kingdom',  # Default country selection
        multi = False,
        style = {'width': '50%'}
    ),

    # Line chart for temperature trends of the selected country
    dcc.Graph(id = 'temperature-line-chart'),

    # World map of average yearly temperatures
    dcc.Graph(
        id = 'temperature-change-map',
        figure = avg_temp_map_fig
    ),

    # Map for changes in average temperature over time
    dcc.Graph(
        id = 'temperature-change-range-map',
        figure = temp_change_map_fig
    )
])

# Callback to update the line chart based on the selected country from the dropdown
@app.callback(
    Output('temperature-line-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):
    # Filter the data for the selected country
    filtered_df = avg_temp_per_year[avg_temp_per_year['Country'] == selected_country]
    
    # Create the line chart for temperature trends
    fig = px.line(filtered_df, x = 'Year', y = 'AverageTemperature',
                  title = f'Average Temperature Trends in {selected_country}',
                  labels = {'Year': 'Year', 'AverageTemperature': 'Average Temperature (°C)'})
    
    # Update the layout for smooth transition
    fig.update_layout(transition_duration=500)


    return fig

app.run_server(debug=True)
